In [25]:
import math

from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger  # noqa
from nltk.corpus import stopwords

In [26]:
# открываем файл

filepath: str = "your filepath"

with open(filepath, "r") as file:
    text: str = file.read().replace("\n", " ")

In [27]:
# подготавливаем текст

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

stopwords = stopwords.words("russian")


def preprocess_text(paragraphs: list[str]) -> list[list[str]]:
    preprocessed = []
    for paragraph in paragraphs:
        doc = Doc(paragraph)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)

        tokens = []
        for token in doc.tokens:
            if token.pos in ("PUNCT", "NUM"):
                continue

            token.lemmatize(morph_vocab)
            lemma = token.lemma.lower()

            if lemma not in stopwords:
                tokens.append(lemma)
        preprocessed.append(tokens)
    return preprocessed

In [28]:
# делим текст на параграфы


def split_text_to_sentences(text: str) -> list[str]:
    doc = Doc(text)
    doc.segment(segmenter)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    return sentences


def group_sentences(
    sentences: list[str],
    group_size: int = 200,
) -> list[str]:
    groups = []
    for i in range(0, len(sentences), group_size):
        group = " ".join(sentences[i : i + group_size])
        groups.append(group)
    return groups


sentences = split_text_to_sentences(text)

group = group_sentences(sentences)

preprocessed_paragraphs = preprocess_text(group)

print("preprocessed_text: ", preprocessed_paragraphs[0][:15])

preprocessed_text:  ['преступление', 'наказание', 'федор', 'михаилович', 'достоевский', 'смерть', 'спасение', 'родион', 'раскольников', 'это', 'роман', 'петербургский', 'студент', 'родион', 'раскольников']


In [29]:
def get_vocabulary(docs: list) -> list[str]:
    vocab_set: set[str] = set()
    for tokens in docs:
        vocab_set.update(tokens)
    vocabulary: list[str] = sorted(list(vocab_set))
    return vocabulary

In [30]:
# bag_of_words


def bag_of_words(text: list[str]) -> list[int]:
    vocabulary = get_vocabulary(text)

    bow_matrix = []
    for tokens in text:
        row = [0] * len(vocabulary)
        for token in tokens:
            if token in vocabulary:
                j = vocabulary.index(token)
                row[j] += 1
        bow_matrix.append(row)

    return bow_matrix


matrix = bag_of_words(preprocessed_paragraphs[:10])
print("\n--- Bag of Words ---")
print("matrix: ", matrix[0][:100])


--- Bag of Words ---
matrix:  [1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 0, 3, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0]


In [31]:
# tf_idf


def compute_df(
    vocabulary: list[str],
    docs: list[list],
) -> list[int]:
    df: list[int] = [0] * len(vocabulary)
    for i, word in enumerate(vocabulary):
        for tokens in docs:
            if word in tokens:
                df[i] += 1
    return df


def compute_idf(
    df: list[int],
    N: int,
) -> list[float]:
    idf: list[float] = [math.log(N / (1 + df_val)) for df_val in df]
    return idf


def compute_tf(
    doc: list[str],
    vocabulary: list[str],
) -> list[float]:
    total_words: int = len(doc)
    word_counts: dict[str, int] = {}
    for token in doc:
        word_counts[token] = word_counts.get(token, 0) + 1

    tf: list[float] = []
    for word in vocabulary:
        count: int = word_counts.get(word, 0)
        tf_val: float = count / total_words if total_words > 0 else 0
        tf.append(tf_val)
    return tf


def compute_tf_idf_for_doc(
    doc: list[str],
    vocabulary: list[str],
    idf: list[float],
) -> list[float]:
    tf: list[float] = compute_tf(doc, vocabulary)
    tf_idf: list[float] = [tf_val * idf_val for tf_val, idf_val in zip(tf, idf)]
    return tf_idf


def tf_idf(docs: list[str]) -> tuple:
    vocabulary = get_vocabulary(docs)
    N = len(docs)
    df = compute_df(vocabulary, docs)
    idf = compute_idf(df, N)

    tfidf_matrix: list[list] = []
    for doc in docs:
        tfidf_vector: list[float] = compute_tf_idf_for_doc(doc, vocabulary, idf)
        tfidf_matrix.append(tfidf_vector)

    return vocabulary, tfidf_matrix


vocab_tfidf, tfidf_matrix = tf_idf(preprocessed_paragraphs)
print("\n--- TF-IDF ---")
print("Словарь:", vocab_tfidf[:100])
print("Матрица TF-IDF:")
print(tfidf_matrix[0][:100])


--- TF-IDF ---
Словарь: ['1789', 'confessions', 'contra', 'danke', 'i', 'ich', 'ii', 'iii', 'iv', 'pro', 'v', 'vi', 'vii', 'xix', 'а-а-а', 'абсолютно', 'авдотья', 'автор', 'адам', 'адвокатский', 'адрес', 'адрес-то', 'адский', 'ажить', 'аккуратно', 'аксиома', 'акцент', 'александр', 'алена', 'алена-то', 'али', 'аллея', 'аль', 'аля', 'амалия', 'амбиция', 'амна', 'ан', 'ана', 'анализ', 'анаполнять', 'ангел', 'англия', 'анна', 'аон', 'апожалеть', 'апосмотреть', 'аппетит', 'арестовать', 'арестовывать', 'арифметика', 'аркадий', 'армяк', 'артельный', 'аршин', 'асердце', 'асессорша', 'асестра', 'ася', 'ата', 'атака', 'атласный', 'атогда', 'атут', 'афанасиевич', 'афанасий', 'афанасий-то', 'африка', 'ах', 'ахать', 'ая', 'б', 'ба', 'баба', 'бабенка', 'бабий', 'бабушка', 'бабушкин', 'багрово-красная', 'багрово-красный', 'бакен', 'балалайка', 'балкон', 'банька', 'барк', 'барышня', 'бастилия', 'батюшка', 'бахрома', 'бахрушин', 'бахус', 'бацилла', 'башмак', 'башмаки-с', 'бег', 'бегать', 'беда', 'бедн